In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pathlib import Path
from dotenv import load_dotenv,find_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.storage import LocalFileStore,InMemoryStore
from langchain_community.vectorstores import FAISS

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
embeddingModelsOAI=OpenAIEmbeddings()
embeddingModelsSTS=SentenceTransformerEmbeddings()

C:\Users\MSI\AppData\Local\Temp\ipykernel_3228\4026543667.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddingModelsSTS=SentenceTransformerEmbeddings()
C:\Users\MSI\AppData\Local\Temp\ipykernel_3228\4026543667.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddingModelsSTS=SentenceTransformerEmbeddings()
c:\Users\MSI\anaconda3\envs\lcnenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipy

<h3>Embed Documents takes in a list of Strings</h3>

In [4]:
embeddings=embeddingModelsOAI.embed_documents(texts=[
                                                "My Son's name is Eklavya",
                                                "He is 11 months old"]
                                             )

In [5]:
embeddings

[[0.0036949492059648037,
  -0.00982087105512619,
  -0.00987905915826559,
  -0.008366166613996029,
  -0.019667604938149452,
  0.04044725000858307,
  -0.027826879173517227,
  -0.008417889475822449,
  -0.015206510201096535,
  0.002102856058627367,
  0.012407012283802032,
  -0.01643492840230465,
  0.02026241645216942,
  -0.0036981820594519377,
  -0.009025633335113525,
  -0.007183007430285215,
  0.020960675552487373,
  -0.02666311524808407,
  0.0288354754447937,
  -0.01072602067142725,
  -0.007881265133619308,
  0.010790674015879631,
  0.003636761102825403,
  -0.004693846218287945,
  -0.0215684175491333,
  0.018387464806437492,
  -0.0031777210533618927,
  -0.01624096743762493,
  0.006982581224292517,
  -0.0010724404128268361,
  -0.0008348387200385332,
  -0.008611850440502167,
  -0.006500912364572287,
  -0.010848862119019032,
  -0.033257775008678436,
  -0.0012146781664341688,
  -0.005172282457351685,
  -0.005582832265645266,
  0.02143911086022854,
  -0.003863048506900668,
  0.020663268864154

In [9]:
len(embeddings)

2

In [6]:
len(embeddings[0])

1536

<h3>Embed Query takes in a single string</h3>

In [7]:
embeddings=embeddingModelsSTS.embed_query(text="My Son's name is Eklavya. He is 6 Months old")

In [8]:
embeddings

[0.0635019838809967,
 -0.016692591831088066,
 -0.04422667995095253,
 0.00529084075242281,
 0.040753550827503204,
 -0.029556747525930405,
 -0.043288033455610275,
 0.0126503836363554,
 0.008146333508193493,
 0.06847063452005386,
 0.02416151389479637,
 -0.05431428179144859,
 0.027749428525567055,
 -0.021238695830106735,
 -0.049221910536289215,
 -0.03571580722928047,
 -0.0027282966766506433,
 -0.018471375107765198,
 0.04253429174423218,
 -0.011691107414662838,
 -0.029468590393662453,
 0.004110620357096195,
 0.007887162268161774,
 0.04725716635584831,
 0.025208979845046997,
 0.030356809496879578,
 -0.04170481488108635,
 0.009100197814404964,
 -0.024130476638674736,
 -0.04726937413215637,
 -0.03173186630010605,
 -0.019016604870557785,
 -0.004384532570838928,
 0.06038268655538559,
 1.4630568330176175e-06,
 0.03868497908115387,
 -0.003407578682526946,
 -0.03151652589440346,
 -0.007072128355503082,
 -0.08192244917154312,
 0.014165708795189857,
 -0.06269326061010361,
 -0.016076834872364998,
 -0.

In [9]:
len(embeddings)

768

In [10]:
embeddingModelsOAI.model

'text-embedding-ada-002'

<h3>Caching of Embeddings</h3>

In [11]:
filePath=Path("data").joinpath("letters").joinpath("state_of_the_union.txt")

In [15]:
tempStore=LocalFileStore(root_path="./cache/")

In [17]:
# The main supported way to initialize a CacheBackedEmbeddings is from_bytes_store
cachedEmbedder=CacheBackedEmbeddings.from_bytes_store(
                                                underlying_embeddings=embeddingModelsOAI,
                                                document_embedding_cache=tempStore, 
                                                namespace="sentenceEmbeddings"
                                                )

In [18]:
# So far empty
list(tempStore.yield_keys())

[]

In [19]:
rawDocument=TextLoader(file_path=filePath,encoding="utf-8").load()
splittedDocuments=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=10).split_documents(documents=rawDocument)

<H2>Creating the Vectorstore</H2>

In [20]:
db=FAISS.from_documents(documents=splittedDocuments,embedding=cachedEmbedder)

In [21]:
# Will not be empty
list(tempStore.yield_keys())

['sentenceEmbeddings01dbc21f-5e4c-5fb5-8d13-517dbe7a32d4',
 'sentenceEmbeddings059eb9ff-c4c8-5ceb-8bf9-0d3d02a92b44',
 'sentenceEmbeddings0fc1ede2-407a-5e14-8f8f-5642214263f5',
 'sentenceEmbeddings1297d37a-2bc1-5e19-bf13-6c950f075062',
 'sentenceEmbeddings17a6727d-8916-54eb-b196-ec9c9d6ca472',
 'sentenceEmbeddings1d203ac6-0e5c-5e99-9c20-884b3206ba18',
 'sentenceEmbeddings20c8f906-bea3-5e8c-b01a-e5ecfa990007',
 'sentenceEmbeddings24610cdc-5e53-577d-8142-5f9c16afbd22',
 'sentenceEmbeddings280a9fbe-0d67-50df-9796-d781d74a0ea4',
 'sentenceEmbeddings2b51a7dd-babe-5c36-8930-3baa92a1130a',
 'sentenceEmbeddings305efb5c-3f01-5657-bcf2-2b92fb1747ca',
 'sentenceEmbeddings3f32125d-8e2c-5e8f-8ea1-0e4fe633c9f5',
 'sentenceEmbeddings3f7b9f1f-79ae-55e3-966a-d0ec952476ed',
 'sentenceEmbeddings464862c8-03d2-5854-b32c-65a075e612a2',
 'sentenceEmbeddings4e5734dd-6470-5d9e-a7b8-468ebc563c4e',
 'sentenceEmbeddings5990258b-0781-5651-8444-c69cb5b6da3d',
 'sentenceEmbeddings5ba09d7e-6a58-5c76-b038-5d8636e5ea25

<h3>Swapping the Bytestore</h3>

In [22]:
memStore=InMemoryStore()

In [23]:
cached_embedder=CacheBackedEmbeddings.from_bytes_store(underlying_embeddings=embeddingModelsSTS,
                                                       document_embedding_cache=memStore,
                                                       namespace="sentenceEmbeddingsMemory")

In [24]:
db=FAISS.from_documents(documents=splittedDocuments,embedding=cachedEmbedder)